In [1]:
import xarray as xr
import numpy as np
from osgeo import gdal
from pathlib import Path
import gc
# rows, cols = 1934, 2242 #we will need to calculate this, see the other notebook
# months = 2 # will be 12

_vars = ['tmin', 'tmax']

import sys 
sys.path.append('Input_production/source/')
import worldclim



In [2]:
out_srs = 'EPSG:6931'
extent_file = '/Users/rwspicer/Desktop/work/TEM/Input_production/working/aoi_5km_buffer_6931.tiff'
from importlib import reload
import rioxarray  # activate the rio accessor
reload(worldclim)

months = list(range(1,13))
print(f'months {months}')
# wc_test = worldclim.WorldClim(months, '', True)
# wc_test.new_from_raster_extent(extent_file)

months [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [ ]:
wc_test = worldclim.WorldClim(
    worldclim.__WORLDCLIM_URL_PATTERN__, 
    extent_file, 
    True, 
    local_location='worldclim',  
    _vars=['tmin']#, 'tmax']#, but remove this line to get all
)

extent (-4602000.0, 4251000.0, 4366000.0, -3485000.0)
resolution, 4000.0,-4000.0
out size 2242, 1934
downloading: https://geodata.ucdavis.edu/climate/worldclim/2_1/base/wc2.1_30s_tmin.zip
